In [20]:
import os
import sys
import pickle
import time
import numpy as np
import pandas as pd
from tqdm.auto  import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import SubsetRandomSampler
from torch_scatter import scatter_mean

from torch_geometric.data import DataLoader
from torch_geometric.nn import (NNConv, GMMConv, GraphConv, Set2Set)
from torch_geometric.data import (InMemoryDataset, Data)
import torch_geometric.transforms as T
from torch_geometric.utils import normalized_cut
# import torch_spline_conv
from torch_geometric.nn import (SplineConv, graclus, max_pool, max_pool_x, global_mean_pool)
# from torch_geometric.nn import graclus,  max_pool, max_pool_x, global_mean_pool
# from torch_spline_conv import (SplineConv)
# import torch_spline_conv

# from torch_geometric.datasets import MNISTSuperpixels
import keras

import progressbar

In [2]:

X_train = np.load("skel_features/mnist/save/X_train.npy", allow_pickle=True)
X_test = np.load("skel_features/mnist/save/X_test.npy", allow_pickle=True)
y_train = np.load("skel_features/mnist/save/y_train.npy", allow_pickle=True)
y_test = np.load("skel_features/mnist/save/y_test.npy", allow_pickle=True)
skeleton_train = np.load("skel_features/mnist/save/skeleton_train.npy", allow_pickle=True)
skeleton_test = np.load("skel_features/mnist/save/skeleton_test.npy", allow_pickle=True)

num_edges = np.max([x.shape[0] for x in np.hstack([skeleton_train, skeleton_test])])
# num_nodes = np.max([len(xzzzzzz) for x in np.hstack([skeleton_train, skeleton_test])])

# for i in range(len(skeleton_train)):
#     tmp = np.zeros((max_len))
#     tmp[:len(skeleton_train[i])] = skeleton_train[i]
#     skeleton_train[i] = tmp
    
# for i in range(len(skeleton_test)):
#     tmp = np.zeros((max_len))
#     tmp[:len(skeleton_test[i])] = skeleton_test[i]
#     skeleton_test[i] = tmp
    
X_train = np.vstack(X_train).reshape(60000,28,28,1)
X_test = np.vstack(X_test).reshape(10000,28,28,1)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
# skeleton_train = np.vstack(skeleton_train)
# skeleton_test = np.vstack(skeleton_test)


num_nodes = 75

### we need to transform skeleton to edge_index

In [3]:
# bar = progressbar.ProgressBar(skeleton_train.shape[0])
# bar.start()

# edge_index_train = []
# for j, skeleton in enumerate(skeleton_train):
#     skeleton = skeleton.reshape(-1,8)
#     nodes_l = skeleton[:, 0:2]
#     nodes_r = skeleton[:, 4:6]
    
#     nodes = np.unique(np.vstack((nodes_l, nodes_r)), axis=0)
#     node_ind = {}
#     for i, node in enumerate(nodes):
#         node_ind[str(node)] = i

#     edge_index = [[],[]]
#     for i in range(nodes_l.shape[0]):
#         node0 = node_ind[str(nodes_l[i])]
#         node1 = node_ind[str(nodes_r[i])]
#         edge_index[0].append(node0)
#         edge_index[1].append(node1)
    
#     edge_index_train.append(edge_index)
    
#     bar.update(j)
    
    

In [4]:
bar = progressbar.ProgressBar(skeleton_test.shape[0])
bar.start()

edge_index_test = []
node_features_test = []
for j, skeleton in enumerate(skeleton_test):
    skeleton = skeleton.reshape(-1,8)
    nodes_l = skeleton[:, 0:2]
    nodes_r = skeleton[:, 4:6]
    
    nodes = np.unique(np.vstack((nodes_l, nodes_r)), axis=0)
    node_ind = {}
    for i, node in enumerate(nodes):
        node_ind[str(node)] = i

    edge_index = [[],[]]
    for i in range(nodes_l.shape[0]):
        node0 = node_ind[str(nodes_l[i])]
        node1 = node_ind[str(nodes_r[i])]
        edge_index[0].append(node0)
        edge_index[1].append(node1)
    
    edge_index_test.append(edge_index)
    
    node_features = np.zeros(num_nodes)
    for skel in skeleton:
        node0 = str(skel[0:2])
        features0 = skel[3]
        node1 = str(skel[4:6])
        features1 = skel[7]
        
        node_features[node_ind[node0]] = features0
        node_features[node_ind[node1]] = features1
        
    node_features_test.append(node_features)
        
    
    bar.update(j)

In [22]:
data = pd.DataFrame(columns=['edge_index', 'node_features'])
data.edge_index = edge_index_test
data.node_features = node_features_test

In [23]:
data.to_csv("test_features.csv")

In [5]:
xx = torch.tensor(node_features_test[5000])
tt = torch.tensor(edge_index_test[5000])

D = Data(x=xx, edge_index=tt)

In [45]:
xx = torch.tensor(node_features_test)
# ei = torch.tensor(edge_index_test)
ei = edge_index_test

D = []
for i in range(xx.shape[0]):
    x = xx[i]
    e = torch.tensor(ei[i])
    D.append(Data(x=x, edge_index=e))
    

In [47]:
len(D)

10000

In [31]:
D

Data(edge_index=[2, 20], x=[64])

In [8]:
class MPNNNet(nn.Module):
    def __init__(self, num_features, aggr='mean', processing_steps=8, message_passing_steps=4, dim=65):
        super(MPNNNet, self).__init__()
        self.message_passing_steps = message_passing_steps
        
        self.lin0 = torch.nn.Linear(num_nodes, dim) # Change 2 to 1 for superpixels
        nn = torch.nn.Sequential(torch.nn.Linear(2, 128), torch.nn.ReLU(), torch.nn.Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr=aggr, root_weight=False)
        self.gru = torch.nn.GRU(dim, dim)
        
        self.set2set = Set2Set(dim, processing_steps=processing_steps)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 10)

    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        h = out.unsqueeze(0)

        for i in range(self.message_passing_steps):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)

        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        return F.log_softmax(self.lin2(out), dim=1)

In [65]:
net = MPNNNet(num_features).double()

In [66]:
net.forward(D)

AttributeError: 'NoneType' object has no attribute 'dim'

### GNNNet

In [211]:
class GNNNet(torch.nn.Module):
    def __init__(self, num_features, num_classes=10):
        super(GNNNet, self).__init__()
        self.conv1 = GraphConv(num_features, 32)
        self.conv2 = GraphConv(32, 64)
        self.conv3 = GraphConv(64, 64)
        self.fc1 = torch.nn.Linear(64, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32, num_classes)
 
    def reset_parameters(self):
        for (name, module) in self._modules.items():
            module.reset_parameters()
 
    def forward(self, data):
        data.x = F.elu(self.conv1(data.x, data.edge_index))
        data.x = F.elu(self.conv2(data.x, data.edge_index))
        data.x = F.elu(self.conv3(data.x, data.edge_index))
#         x_1 = scatter_mean(data.x, data.batch, dim=0)
#         x = x_1
        x = data.x
 
        x = F.elu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.elu(self.fc2(x))
        x = self.fc3(x)
        print(x)
        return F.log_softmax(x)

 ### 1 object

In [256]:
gnn = GNNNet(num_features=num_nodes).double()

In [257]:
xx = torch.tensor(node_features_test[5000])
tt = torch.tensor(edge_index_test[5000])

D = Data(x=xx, edge_index=tt)

In [259]:
gnn.forward(D)

RuntimeError: size mismatch, m1: [1 x 64], m2: [75 x 32] at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/TH/generic/THTensorMath.cpp:136

### training

In [195]:
xx = torch.tensor(node_features_test)
# ei = torch.tensor(edge_index_test)
ei = edge_index_test

D = []
for i in range(xx.shape[0]):
    x = xx[i]
    e = torch.tensor(ei[i])
    D.append(Data(x=x, edge_index=e))
    
data = pd.DataFrame(columns=['col', 'y'])
data.col=D
data.y = torch.tensor(y_test)

In [196]:
gnn = GNNNet(num_features=num_nodes).double()

In [197]:
# gnn.forward(D)

In [198]:

indices = list(range(len(D)))
np.random.seed(43)
np.random.shuffle(indices)
train_indices = indices[:100]
sampler = SubsetRandomSampler(train_indices)
train_loader = DataLoader(D, batch_size=1, sampler=sampler, shuffle=False)

In [199]:
optimizer = torch.optim.Adam(gnn.parameters(), lr=0.01)

In [200]:
D[0]

Data(edge_index=[2, 39], x=[75])

In [255]:
gnn.train() 

for dat in train_loader:
    optimizer.zero_grad()
    response = gnn.forward(dat)
    print(response)
    print(response.dim())
    F.nll_loss(response, dat.y).backward()
    optimizer.step()

RuntimeError: index out of range: Tried to access index 41 out of table with 31 rows. at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:418

In [75]:
def train(model, train_loader, optimizer):
    model.train()

    for data in tqdm(train_loader, leave=False):
#         data = data.to(device)
        optimizer.zero_grad()
        responce = model(data)
        F.nll_loss(responce, data.y).backward()
        optimizer.step()
        

In [76]:
train(gnn, train_loader, optimizer)

RuntimeError: size mismatch, m1: [1 x 4800], m2: [75 x 32] at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/TH/generic/THTensorMath.cpp:136